In [32]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Simulation settings
dtime = 0.01 # time step
iterative = True

# Capacitor
capacitance = 10e-6 # farads
phase = np.pi / 2 # 90 degrees, TODO: change to calculate based on resistance

# Zener diode
has_zener = True

# Resistor
resistance = 200 # ohm

# Source
frequency = 1 # hertz
voltage = 5 # volts
current_volt = lambda t: np.sin(2*np.pi*t*frequency)# * voltage # function given time in seconds

# Rectifiers

# False is single diode, True is diode bridge
diode_bridge = True

if diode_bridge:
    rect_volt = lambda volt: np.abs(volt)
else:
    rect_volt = lambda volt: volt if volt >= 1e-14 else 0


In [ ]:
# Calculating functions
def charge_capacitor(time_step, current_charge, applied_volt):
    time_const = capacitance * resistance
    
    next_charge = current_charge - applied_volt

In [ ]:
if iterative:
    

In [34]:
for t in range(10): print(t/4, rect_volt(current_volt(t/4)))

0.0 0.0
0.25 1.0
0.5 1.2246467991473532e-16
0.75 1.0
1.0 2.4492935982947064e-16
1.25 1.0
1.5 3.6739403974420594e-16
1.75 1.0
2.0 4.898587196589413e-16
2.25 1.0
